In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import numpy as np
import os
import gzip

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
# print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

(20640, 8)
(20640,)


In [18]:
from sklearn.model_selection import train_test_split
# 注意housing本身的转变 以及train_test_split参数设置
x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=7)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
print(len(x_train_scaled), len(x_train))
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)
11610 11610


In [6]:
def draw_learning_curve(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid()
    plt.ylim(0, 1)
    plt.show()

 fit函数做了什么？
1、以batch的形式遍历训练集  ->metric，损失函数
   1.1 自动求导
2、epoch结束，验证集验证   ->metric

要改动1.1 需要将封装好的函数展开

In [14]:
# metric使用
metric = keras.metrics.MeanSquaredError()
#          y_pred  y_true
print(metric([5.], [2.]))
print(metric([0.], [1.]))
print(metric([7.], [5.]))
print(metric([7.], [5.]))
# metric会自动累加，取平均
print(metric.result())

# epoch结束，一般需要关掉累加，就是重置metric
metric.reset_states()
print(metric([1.], [3.]))
print(metric.result())

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(5.0, shape=(), dtype=float32)
tf.Tensor(4.6666665, shape=(), dtype=float32)
tf.Tensor(4.5, shape=(), dtype=float32)
tf.Tensor(4.5, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(4.0, shape=(), dtype=float32)


In [ ]:
epochs = 100
batch_size = 32
steps_per_epoch = len(x_train_scaled) // batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()
# 随即遍历，fit函数中随机遍历数据：shuffle + 遍历
# 简化版，随机取
def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x_train), size=batch_size)  # len(x_train)=x_train.shape[0]
    return x[idx], y[idx]

# 建立模型
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=x_train.shape[1:]),
    keras.layers.Dense(1)
])

# 代替fit
for epoch in range(epochs):
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch, y_batch = random_batch(x_train_scaled, y_train, batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)      # (32,1)
            # (32, 1) ↑    loss.shape=()  ↓ 函数
            loss = tf.reduce_mean(keras.losses.mean_squared_error(y_batch, y_pred)) # 32 -> 1  
            metric(y_batch, y_pred)
        grads = tape.gradient(loss, model.variables)
        # 绑定梯度和变量
        grads_and_vars = zip(grads, model.variables)
        optimizer.apply_gradients(grads_and_vars)
        print("\rEpoch", epoch, "train mse:", metric.result().numpy(), end="")
    y_valid_pred = model(x_valid_scaled)
    valid_loss = tf.reduce_mean(keras.losses.mean_squared_error(y_valid_pred, y_valid))
    print("\t", "valid mse:", valid_loss.numpy())

In [ ]:
# 无法执行 draw_learning_curve(history)
# 因为没有history